<a href="https://colab.research.google.com/github/tijeco/berteome/blob/14-esm-hugging-face/03_general_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.7 MB/s 
     |████████████████████████████████| 182 kB 49.9 MB/s 
     |████████████████████████████████| 7.6 MB 43.9 MB/s 
